In [1]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.7:4043
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1641126818129)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@2f46ef08


앞 장에서는 DataFrame의 기본 개념과 DataFrame의 핵심 추상화 개념을 알아보았음<br/>
이 장에서는 스파크의 구조적 연산에서 가장 중요한 내용인 표현식을 만드는 방법을 알아보겠음<br/>
그리고 다음과 같은 다양한 데이터 타입을 다루는 방법도 함께 알아볼 것임<br/>
* Boolean 타입
* 수치 타입
* 문자열 타입
* date와 timestamp 타입
* null 값 다루기
* 복합 데이터 타입
* 사용자 정의 함수

# 6.1 API는 어디서 찾을까

스파크는 현재 활발하게 성장 중인 프로젝트이고, 이 책을 포함한 모든 스파크 관련 서적 내용은 언젠가 예전 버전의 내용이 될 것임<br/>
그러므로 데이터 변환용 함수를 어떻게 찾는지 알아야 함 (*의문사항: 지금 내용이 outdate되는 것과 데이터 변환용 함수 어떻게 찾는지 아는 것이 무슨 상관?*)<br/>
데이터 변환용 함수를 찾기 위해 핵심적으로 보아야 할 부분은 다음과 같음<br/>

* DataFrame(Dataset) 메서드
DataFrame은 Row 가진 Dataset이므로 결국에는 Dataset 메서드를 만나게 됨<br/>
*DataFrameStatFunctions*와 *DataFrameNaFunctions* 등 Dataset의 하위 모듈은 다양한 메서드를 제공함<br/>
이 메서드를 사용해 여러 가지 문제를 해결할 수 있음<br/>
예를 들어 *DataFrameStatFunctions*는 다양한 통계적 함수를 제공하며, *DataFrameNaFunctions*는 null 데이터를 다루는 데 필요한 함수를 제공함<br/>

* Column 메서드
5장에서 컬럼 관련 메서드를 알아보았음<br/>
Column은 alias나 contains 같이 컬럼과 관련된 여러 가지 메서드를 제공함<br/>
org.apache.spark.sql.functions 패키지는 데이터 타입과 관련된 다양한 함수를 제공함<br/>
이 패키지는 많이 쓰이므로 보통 전체 패키지를 import하고 사용함<br/>

모든 함수는 데이터 로우의 특정 포맷이나 구조를 다른 형태로 변환하기 위해 존재함<br/>
함수를 사용해 더 많은 로우를 만들거나 줄일 수 있음<br/>
다음은 분석에 사용할 DataFrame을 생성하는 예제임<br/>

In [5]:
val df = spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("Downloads/Spark-The-Definitive-Guide/data/retail-data/by-day/2010-12-01.csv")
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



df: org.apache.spark.sql.DataFrame = [InvoiceNo: string, StockCode: string ... 6 more fields]


In [9]:
df.createOrReplaceTempView("dfTable")
spark.sql("SELECT * FROM dfTable").show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|     7.65|   17850.0|United Kingdom|
|   536365|    21730|GLASS S

*df.createOrReplaceTempView("view_name")이란?* <br/>
스파크에서 DataFrame API 말고 SparkSession.sql() 메서드를 통해 SQL문으로 데이터를 다룰 수 있음<br/>
sql 메서드도 컴파일 과정에서 DataFrame API 꼴로 변환되니 df.select("col1", "col2").show()처럼 DataFrame API를 쓰는 경우와 같다고 볼 수 있음<br/>
하지만 둘의 차이점이 있으니, SQL문으로 작업하려면 df의 View를 생성해야 한다는 것임<br/>
그것이 df.createOrReplaceTempView("view_name") 이 부분임<br/>

이 부분은 [블로그](https://jhleeeme.github.io/spark-temp-view/) 설명을 참고함<br/>

# 6.2 스파크 데이터 타입으로 변환하기

프로그래밍 언어의 고유 데이터 타입을 스파크 데이터 타입으로 변환해보겠음<br/>
스파크 데이터 타입으로 변환하는 방법은 반드시 알아두어야 함<br/>
데이터 타입 변환은 *lit* 함수를 사용함<br/>
lit 함수는 다른 언어의 데이터 타입을 스파크 데이터 타입에 맞게 변환함<br/>
스칼라의 다양한 데이터 타입을 스파크 데이터 타입으로 변환해보겠음<br/>

In [10]:
import org.apache.spark.sql.functions.lit

df.select(lit(5), lit("five"), lit(5.0))

import org.apache.spark.sql.functions.lit
res9: org.apache.spark.sql.DataFrame = [5: int, five: string ... 1 more field]


# 6.3 불리언 데이터 타입 다루기

불리언은 모든 필터링 작업의 기반이므로 데이터 분석에 필수적임<br/>
불리언 구문은 and, or, true, false로 구성됨<br/>
불리언 구문을 사용해 true 또는 false로 평가되는 논리 문법을 만듦<br/>
논리 문법은 데이터 로우를 필터링할 때 필요조건의 일치(true)와 불일치(false)를 판별하는 데 사용됨<br/>

소매 데이터셋을 사용해 불리언을 다루어보겠음<br/>
불리언 식에는 일치 조건뿐만 아니라 작다, 크다와 같은 비교 연산 조건을 사용할 수 있음<br/>

In [11]:
import org.apache.spark.sql.functions.col

df.where(col("InvoiceNo").equalTo(536365))
    .select("InvoiceNo", "Description")
    .show(5, false) // true는 내용이 길면 ...로 표시하는 것이고 false로 하면 풀네임으로 표시됨

+---------+-----------------------------------+
|InvoiceNo|Description                        |
+---------+-----------------------------------+
|536365   |WHITE HANGING HEART T-LIGHT HOLDER |
|536365   |WHITE METAL LANTERN                |
|536365   |CREAM CUPID HEARTS COAT HANGER     |
|536365   |KNITTED UNION FLAG HOT WATER BOTTLE|
|536365   |RED WOOLLY HOTTIE WHITE HEART.     |
+---------+-----------------------------------+
only showing top 5 rows



import org.apache.spark.sql.functions.col


In [12]:
import org.apache.spark.sql.functions.col

df.where(col("InvoiceNo") === 536365)
    .select("InvoiceNo", "Description")
    .show(5, false) // true는 내용이 길면 ...로 표시하는 것이고 false로 하면 풀네임으로 표시됨

+---------+-----------------------------------+
|InvoiceNo|Description                        |
+---------+-----------------------------------+
|536365   |WHITE HANGING HEART T-LIGHT HOLDER |
|536365   |WHITE METAL LANTERN                |
|536365   |CREAM CUPID HEARTS COAT HANGER     |
|536365   |KNITTED UNION FLAG HOT WATER BOTTLE|
|536365   |RED WOOLLY HOTTIE WHITE HEART.     |
+---------+-----------------------------------+
only showing top 5 rows



import org.apache.spark.sql.functions.col


가장 명확한 방법은 문자열 표현식에 조건절을 명시하는 것임<br/>
이 방법으로 다음과 같이 일치 여부를 표현할 수 있음<br/>

In [13]:
df.where("InvoiceNo = 536365").show(5, false)

+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |2010-12-01 08:26:00|2.55     |17850.0   |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |2010-12-01 08:26:00|2.75     |17850.0   |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
+---------+-----

In [14]:
df.where("InvoiceNo <> 536365").show(5, false)

+---------+---------+-----------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                  |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------+--------+-------------------+---------+----------+--------------+
|536366   |22633    |HAND WARMER UNION JACK       |6       |2010-12-01 08:28:00|1.85     |17850.0   |United Kingdom|
|536366   |22632    |HAND WARMER RED POLKA DOT    |6       |2010-12-01 08:28:00|1.85     |17850.0   |United Kingdom|
|536367   |84879    |ASSORTED COLOUR BIRD ORNAMENT|32      |2010-12-01 08:34:00|1.69     |13047.0   |United Kingdom|
|536367   |22745    |POPPY'S PLAYHOUSE BEDROOM    |6       |2010-12-01 08:34:00|2.1      |13047.0   |United Kingdom|
|536367   |22748    |POPPY'S PLAYHOUSE KITCHEN    |6       |2010-12-01 08:34:00|2.1      |13047.0   |United Kingdom|
+---------+---------+-----------------------------+--------+----

and 메서드나 or 메서드를 사용해서 불리언 표현식을 여러 부분에 지정할 수 있음<br/>
불리언 표현식을 사용하는 경우 항상 모든 표현식을 and 메서드로 묶어 차례대로 필터를 적용해야 함<br/>

차례대로 필터를 적용해야 하는 이유는 다음과 같음<br/>
불리언 문을 차례대로 표현하더라도 스파크는 내부적으로 and 구문을 필터 사이에 추가해 모든 필터를 하나의 문장으로 변환함<br/>
그런 다음 동시에 모든 필터를 처리함<br/>
원한다면 and 구문으로 조건문을 만들 수도 있음<br/>
하지만 차례로 조건을 나열하면 이해하기 쉽고 읽기도 편해짐<br/>
반면 or 구문을 사용할 때는 반드시 동일한 구문에 조건을 정의해야 함<br/>

In [15]:
val priceFilter = col("UnitPrice") > 600
val descripFilter = col("Description").contains("POSTAGE")
df.where(col("StockCode").isin("DOT")).where(priceFilter.or(descripFilter)).show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



priceFilter: org.apache.spark.sql.Column = (UnitPrice > 600)
descripFilter: org.apache.spark.sql.Column = contains(Description, POSTAGE)


불리언 표현식을 필터링 조건에만 사용하는 것은 아님<br/>
불리언 컬럼을 사용해 DataFrame을 필터링할 수도 있음<br/>

In [17]:
val DOTCodeFilter = col("StockCode") === "DOT"
val priceFilter = col("UnitPrice") > 600
val descripFilter = col("Description").contains("POSTAGE")

df.withColumn("isExpensive", DOTCodeFilter.and(priceFilter.or(descripFilter)))
    .where("isExpensive")
    .select("unitPrice", "isExpensive").show(5)

+---------+-----------+
|unitPrice|isExpensive|
+---------+-----------+
|   569.77|       true|
|   607.49|       true|
+---------+-----------+



DOTCodeFilter: org.apache.spark.sql.Column = (StockCode = DOT)
priceFilter: org.apache.spark.sql.Column = (UnitPrice > 600)
descripFilter: org.apache.spark.sql.Column = contains(Description, POSTAGE)


*의문사항*<br/>
위의 예제 코드에서 where("isExpensive") 부분이 무슨 역할을 하는지?<br/>
해당 부분을 빼도 작동하지 않을까?<br/>

In [19]:
//위의 의문사항을 해결하기 위해 where("isExpensive") 부분을 빼고 돌려보는 코드

val DOTCodeFilter = col("StockCode") === "DOT"
val priceFilter = col("UnitPrice") > 600
val descripFilter = col("Description").contains("POSTAGE")

df.withColumn("isExpensive", DOTCodeFilter.and(priceFilter.or(descripFilter)))
    .select("unitPrice", "isExpensive").show(5)

+---------+-----------+
|unitPrice|isExpensive|
+---------+-----------+
|     2.55|      false|
|     3.39|      false|
|     2.75|      false|
|     3.39|      false|
|     3.39|      false|
+---------+-----------+
only showing top 5 rows



DOTCodeFilter: org.apache.spark.sql.Column = (StockCode = DOT)
priceFilter: org.apache.spark.sql.Column = (UnitPrice > 600)
descripFilter: org.apache.spark.sql.Column = contains(Description, POSTAGE)


*의문사항 해결*<br/>
where("isExpensive") 부분을 빼고 돌리니까 isExpensive가 true냐 false냐를 구분하지 않고 상위 5개 로우가 그대로 출력됐음<br/>
즉 where("isExpensive") 부분의 역할은 isExpensive 컬럼값이 true인 로우만 필터링하는 것이었음<br/>

필터를 반드시 표현식으로 정의할 필요는 없음<br/>
별도의 작업 없이 컬럼명을 사용해 필터를 정의할 수도 있음<br/>

이 모든 구문을 where절로 표현할 수 있음<br/>
그리고 필터를 표현해야 한다면 DataFrame 인터페이스 방식보다 SQL이 훨씬 쉬움<br/>
스파크 SQL을 사용한다고 해서 성능 저하가 발생하는 것은 아님<br/>
예를 들어 다음 두 문장은 동일하게 처리됨<br/>

In [20]:
import org.apache.spark.sql.functions.{expr, not, col}

df.withColumn("isExpensive", not(col("UnitPrice").leq(250)))
    .filter("isExpensive")
    .select("Description", "UnitPrice").show(5)

+--------------+---------+
|   Description|UnitPrice|
+--------------+---------+
|DOTCOM POSTAGE|   569.77|
|DOTCOM POSTAGE|   607.49|
+--------------+---------+



import org.apache.spark.sql.functions.{expr, not, col}


In [21]:
df.withColumn("isExpensive", expr("NOT UnitPrice <= 250"))
    .filter("isExpensive")
    .select("Description", "UnitPrice").show(5)

+--------------+---------+
|   Description|UnitPrice|
+--------------+---------+
|DOTCOM POSTAGE|   569.77|
|DOTCOM POSTAGE|   607.49|
+--------------+---------+



'불리언 표현식을 만들 때 null 값 데이터는 어떻게 다뤄야 하지?'라는 의문이 들 것임<br/>
null 값 데이터는 조금 다르게 처리해야 함<br/>
그런 경우엔 null 값에 안전한(null-safe) 동치(equivalence)를 사용하면 됨<br/>
그러면 null 값을 가진 경우 어떤 값과의 일치 여부를 test할 때 null값을 반환하지 않고 true 혹은 false 값을 반환하게 됨<br/>
자세한 설명은 [여기](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.Column.eqNullSafe.html)를 참고할 것<br/>

# 6.4 수치형 데이터 타입 다루기

count는 빅데이터 처리에서 필터링 다음으로 많이 수행하는 작업임<br/>
대부분은 수치형 데이터 타입을 사용해 연산 방식을 정의하기만 하면 됨<br/>

상황을 가정해 가상의 예제를 만들어보겠음<br/>
우리는 소매 데이터셋의 수량을 잘못 기록했고, 실제 수량은 (현재 수량 * 단위 가격)^2 + 5 공식으로 구할 수 있다는 사실을 알았음<br/>
이 문제를 해결하려면 수치형 함수인 pow를 사용함<br/>
pow 함수는 표시된 지수만큼 컬럼의 값을 거듭제곱함<br/>

In [26]:
import org.apache.spark.sql.functions.{expr, pow}

val fabricatedQuantity = pow(col("Quantity") * col("UnitPrice"), 2) + 5
df.select(expr("CustomerId"), fabricatedQuantity.alias("realQuantity")).show(10)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
|   17850.0|             489.0|
|   17850.0|          418.7156|
|   17850.0|          418.7156|
|   17850.0|239.09000000000003|
|   17850.0|            655.25|
|   17850.0|128.21000000000004|
|   17850.0|128.21000000000004|
|   13047.0|2929.6463999999996|
+----------+------------------+
only showing top 10 rows



import org.apache.spark.sql.functions.{expr, pow}
fabricatedQuantity: org.apache.spark.sql.Column = (POWER((Quantity * UnitPrice), 2.0) + 5)


위 예제의 두 컬럼 모두 수치형이므로 곱셈 연산이 가능함<br/>
그리고 필요한 경우 덧셈이나 뺄셈도 가능함<br/>

In [27]:
df.selectExpr(
    "CustomerId",
    "(POWER((Quantity * UnitPrice), 2.0) + 5) as realQuantity").show(10)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
|   17850.0|             489.0|
|   17850.0|          418.7156|
|   17850.0|          418.7156|
|   17850.0|239.09000000000003|
|   17850.0|            655.25|
|   17850.0|128.21000000000004|
|   17850.0|128.21000000000004|
|   13047.0|2929.6463999999996|
+----------+------------------+
only showing top 10 rows



반올림도 자주 사용하는 수치형 작업 중 하나임<br/>
때로는 소수점 자리를 없애기 위해 Integer 타입으로 형변환하기도 함<br/>
하지만 스파크는 정확한 계산이 가능한 함수를 제공함<br/>
그리고 정밀도를 사용해 더 세밀한 작업을 수행할 수 있음<br/>
다음은 소수점 첫째 자리에서 반올림하는 예제임<br/>

In [28]:
import org.apache.spark.sql.functions.{round, bround}

df.select(round(col("UnitPrice"), 1).alias("rounded"), col("UnitPrice")).show(10)

+-------+---------+
|rounded|UnitPrice|
+-------+---------+
|    2.6|     2.55|
|    3.4|     3.39|
|    2.8|     2.75|
|    3.4|     3.39|
|    3.4|     3.39|
|    7.7|     7.65|
|    4.3|     4.25|
|    1.9|     1.85|
|    1.9|     1.85|
|    1.7|     1.69|
+-------+---------+
only showing top 10 rows



import org.apache.spark.sql.functions.{round, bround}


기본적으로 round 함수는 소수점 값이 정확히 중간값 이상이라면 반올림함<br/>
내림은 bround 함수를 사용함<br/>

In [33]:
import org.apache.spark.sql.functions.lit

df.select(round(lit("2.5")), bround(lit("2.5"))).show(2)

+-------------+--------------+
|round(2.5, 0)|bround(2.5, 0)|
+-------------+--------------+
|          3.0|           2.0|
|          3.0|           2.0|
+-------------+--------------+
only showing top 2 rows



import org.apache.spark.sql.functions.lit


두 컬럼 사이의 상관관계를 계산하는 것도 수치형 연산 작업 중 하나임<br/>
예를 들어 고객이 비싼 물건보다 저렴한 물건을 더 많이 구매하는지 알기 위해 두 컬럼에 대한 피어슨 상관계수를 계산해볼 필요가 있음<br/>
다음 예제와 같이 DataFrame의 통계용 함수나 메서드를 사용해 피어슨 상관계수를 계산할 수 있음<br/>

In [42]:
import org.apache.spark.sql.functions.{corr}

df.stat.corr("Quantity", "UnitPrice")
df.select(corr("Quantity", "UnitPrice")).show()

+-------------------------+
|corr(Quantity, UnitPrice)|
+-------------------------+
|     -0.04112314436835551|
+-------------------------+



import org.apache.spark.sql.functions.corr


하나 이상의 컬럼에 대한 요약 통계를 계산하는 작업 역시 자주 수행됨<br/>
요약 통계는 describe 메서드를 사용해 얻을 수 있음<br/>
describe 메서드는 관련 컬럼에 대한 집계(count), 평균(mean), 표준편차(stddev), 최솟값(min), 최댓값(max)을 계산함<br/>
통계 스키마는 변경될 수 있으므로 describe 메서드는 콘솔 확인용으로만 사용해야 함<br/>

In [47]:
df.select(col("Quantity"), col("UnitPrice"), col("CustomerID")).describe().show()

+-------+------------------+------------------+------------------+
|summary|          Quantity|         UnitPrice|        CustomerID|
+-------+------------------+------------------+------------------+
|  count|              3108|              3108|              1968|
|   mean| 8.627413127413128| 4.151946589446603|15661.388719512195|
| stddev|26.371821677029203|15.638659854603892|1854.4496996893627|
|    min|               -24|               0.0|           12431.0|
|    max|               600|            607.49|           18229.0|
+-------+------------------+------------------+------------------+



정확한 수치가 필요하다면 함수를 import하고 해당 컬럼에 적용하는 방식으로 직접 집계를 수행할 수 있음<br/>


In [48]:
import org.apache.spark.sql.functions.{count, mean, stddev_pop, min, max}

import org.apache.spark.sql.functions.{count, mean, stddev_pop, min, max}


StatFunctions 패키지는 다양한 통계 함수를 제공함<br/>
stat 속성을 사용해 접근할 수 있으며 다양한 통곗값을 계산할 때 사용하는 DataFrame 메서드임<br/>
예컨대 approxQuantile 메서드를 사용해 데이터의 백분위수를 정확하게 계산하거나 근사치를 계산할 수 있음<br/>

In [49]:
val colName = "UnitPrice"
val quantileProbs = Array(0.5)
val relError = 0.05

df.stat.approxQuantile("UnitPrice", quantileProbs, relError) // 2.51

colName: String = UnitPrice
quantileProbs: Array[Double] = Array(0.5)
relError: Double = 0.05
res47: Array[Double] = Array(2.51)


StatFunctions 패키지는 교차표(cross-tabulation)나 자주 사용하는 항목 쌍을 확인하는 용도의 메서드도 제공함<br/>
단, 연산 결과가 너무 크면 화면에 모두 보이지 않을 수 있음<br/>

In [56]:
df.stat.crosstab("StockCode", "Quantity").show()

+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|StockCode_Quantity| -1|-10|-12| -2|-24| -3| -4| -5| -6| -7|  1| 10|100| 11| 12|120|128| 13| 14|144| 15| 16| 17| 18| 19|192|  2| 20|200| 21|216| 22| 23| 24| 25|252| 27| 28|288|  3| 30| 32| 33| 34| 36|384|  4| 40|432| 47| 48|480|  5| 50| 56|  6| 60|600| 64|  7| 70| 72|  8| 80|  9| 96|
+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|             22578|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0| 

*의문사항*<br/>
crosstab이란 것이 무엇이고 왜 쓰는 것인지?<br/>

In [57]:
df.stat.freqItems(Seq("StockCode", "Quantity")).show()

+--------------------+--------------------+
| StockCode_freqItems|  Quantity_freqItems|
+--------------------+--------------------+
|[90214E, 20728, 2...|[200, 128, 23, 32...|
+--------------------+--------------------+



마지막으로 소개할 StatFunctions 패키지의 함수는 monotonically_increasing_id임<br/>
이 함수는 모든 로우에 고유 ID 값을 추가함<br/>
이 함수는 모든 로우에 0부터 시작하는 고윳값을 생성함<br/>

In [65]:
import org.apache.spark.sql.functions.monotonically_increasing_id

df.select(monotonically_increasing_id(), col("Quantity"), col("UnitPrice"), col("CustomerID")).show(20)

+-----------------------------+--------+---------+----------+
|monotonically_increasing_id()|Quantity|UnitPrice|CustomerID|
+-----------------------------+--------+---------+----------+
|                            0|       6|     2.55|   17850.0|
|                            1|       6|     3.39|   17850.0|
|                            2|       8|     2.75|   17850.0|
|                            3|       6|     3.39|   17850.0|
|                            4|       6|     3.39|   17850.0|
|                            5|       2|     7.65|   17850.0|
|                            6|       6|     4.25|   17850.0|
|                            7|       6|     1.85|   17850.0|
|                            8|       6|     1.85|   17850.0|
|                            9|      32|     1.69|   13047.0|
|                           10|       6|      2.1|   13047.0|
|                           11|       6|      2.1|   13047.0|
|                           12|       8|     3.75|   13047.0|
|       

import org.apache.spark.sql.functions.monotonically_increasing_id


불규칙적으로 데이터를 생성할 수 있는 rand() 함수나 randn() 함수 등 임의 데이터 생성 함수가 있음<br/>
이 함수들은 잠재적으로 결정론(determinism)과 관련된 문제를 가지고 있음<br/>
스파크 메일링 리스트에서 이와 관련된 논의를 찾아볼 수 있음<br/>
또한 최신 버전의 StatFunction 패키지는 블룸(bloom) 필터링이나 sketching algorithms 같은 여러 고급 기법과 관련된 함수를 제공함<br/>
더 자세한 내용은 [API 문서](http://bit.ly/2ptAiY2)를 참고할 것<br/>

# 6.5 문자열 데이터 타입 다루기

문자열을 다루는 작업은 거의 모든 데이터 처리 과정에서 발생함<br/>
그러므로 문자열을 다루는 방법을 알아야 함<br/>
로그 파일에 정규 표현식을 사용해 데이터 추출, 데이터 치환, 문자열 존재 여부, 대소문자 변환 처리 등의 작업을 할 수 있음<br/>

대소문자 변환 작업부터 시작하겠음<br/>
initcap 함수는 주어진 문자열에서 공백으로 나뉘는 모든 단어의 첫 글자를 대문자로 변경함<br/>

In [67]:
import org.apache.spark.sql.functions.{initcap}

df.select(initcap(col("Description"))).show(false)

+-----------------------------------+
|initcap(Description)               |
+-----------------------------------+
|White Hanging Heart T-light Holder |
|White Metal Lantern                |
|Cream Cupid Hearts Coat Hanger     |
|Knitted Union Flag Hot Water Bottle|
|Red Woolly Hottie White Heart.     |
|Set 7 Babushka Nesting Boxes       |
|Glass Star Frosted T-light Holder  |
|Hand Warmer Union Jack             |
|Hand Warmer Red Polka Dot          |
|Assorted Colour Bird Ornament      |
|Poppy's Playhouse Bedroom          |
|Poppy's Playhouse Kitchen          |
|Feltcraft Princess Charlotte Doll  |
|Ivory Knitted Mug Cosy             |
|Box Of 6 Assorted Colour Teaspoons |
|Box Of Vintage Jigsaw Blocks       |
|Box Of Vintage Alphabet Blocks     |
|Home Building Block Word           |
|Love Building Block Word           |
|Recipe Box With Metal Heart        |
+-----------------------------------+
only showing top 20 rows



import org.apache.spark.sql.functions.initcap


lower 함수를 사용해 문자열 전체를 소문자로 변경하거나, upper 함수를 사용해 문자열 전체를 대문자로 변경할 수 있음<br/>

In [69]:
import org.apache.spark.sql.functions.{upper, lower}

df.select(col("Description"),
         lower(col("Description")),
         upper(lower(col("Description")))).show(false)

+-----------------------------------+-----------------------------------+-----------------------------------+
|Description                        |lower(Description)                 |upper(lower(Description))          |
+-----------------------------------+-----------------------------------+-----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER |white hanging heart t-light holder |WHITE HANGING HEART T-LIGHT HOLDER |
|WHITE METAL LANTERN                |white metal lantern                |WHITE METAL LANTERN                |
|CREAM CUPID HEARTS COAT HANGER     |cream cupid hearts coat hanger     |CREAM CUPID HEARTS COAT HANGER     |
|KNITTED UNION FLAG HOT WATER BOTTLE|knitted union flag hot water bottle|KNITTED UNION FLAG HOT WATER BOTTLE|
|RED WOOLLY HOTTIE WHITE HEART.     |red woolly hottie white heart.     |RED WOOLLY HOTTIE WHITE HEART.     |
|SET 7 BABUSHKA NESTING BOXES       |set 7 babushka nesting boxes       |SET 7 BABUSHKA NESTING BOXES       |
|GLASS STA

import org.apache.spark.sql.functions.{upper, lower}


문자열 주변의 공백을 제거하거나 추가하는 작업도 가능함<br/>
이 작업은 lpad, ltrim, rpad, rtrim, trim 함수를 사용함<br/>

In [73]:
import org.apache.spark.sql.functions.{lit, ltrim, rtrim, lpad, rpad, trim}

df.select(
    ltrim(lit("    HELLO    ")).as("ltrim"),
    rtrim(lit("    HELLO    ")).as("rtrim"),
    trim(lit("    HELLO    ")).as("trim"),
    lpad(lit("HELLO"), 10, " ").as("lp"),
    rpad(lit("HELLO"), 10, " ").as("rp")).show(2)    

+---------+---------+-----+----------+----------+
|    ltrim|    rtrim| trim|        lp|        rp|
+---------+---------+-----+----------+----------+
|HELLO    |    HELLO|HELLO|     HELLO|HELLO     |
|HELLO    |    HELLO|HELLO|     HELLO|HELLO     |
+---------+---------+-----+----------+----------+
only showing top 2 rows



import org.apache.spark.sql.functions.{lit, ltrim, rtrim, lpad, rpad, trim}


*유의할 점*<br/>
lpad 함수나 rpad 함수에 문자열의 길이보다 작은 숫자를 넘기면 문자열의 오른쪽부터 제거됨<br/>

## 6.5.1 정규 표현식

문자열의 존재 여부를 확인하거나 일치하는 모든 문자열을 치환할 때는 보통 **정규 표현식**을 사용함<br/>
정규 표현식을 사용해 문자열에서 값을 추출하거나 다른 값으로 치환하는 데 필요한 규칙 모음을 정의할 수 있음<br/>

스파크는 자바 정규 표현식이 가진 강력한 능력을 활용함<br/>
자바 정규 표현식 문법은 보통 사용하는 언어의 문법과 약간 다르므로 운영 환경에서 정규 표현식을 사용하기 전에 다시 한 번 검토해야 함<br/>
스파크는 정규 표현식을 위해 regexp_extract 함수와 regexp_replace 함수를 제공함<br/>
이 함수들은 값을 추출하고 치환하는 역할을 함<br/>

regexp_replace 함수를 사용해 'description' 컬럼의 값을 'COLOR'로 치환해보겠음<br/>

In [76]:
import org.apache.spark.sql.functions.regexp_replace

val simpleColors = Seq("black", "white", "red", "green", "blue")
val regexString = simpleColors.map(_.toUpperCase).mkString("|")
// 파이프 문자(|)는 정규 표현식에서 OR를 의미
df.select(
    regexp_replace(col("Description"), regexString, "COLOR").alias("color_clean"),
    col("Description")).show(2, false)

+----------------------------------+----------------------------------+
|color_clean                       |Description                       |
+----------------------------------+----------------------------------+
|COLOR HANGING HEART T-LIGHT HOLDER|WHITE HANGING HEART T-LIGHT HOLDER|
|COLOR METAL LANTERN               |WHITE METAL LANTERN               |
+----------------------------------+----------------------------------+
only showing top 2 rows



import org.apache.spark.sql.functions.regexp_replace
simpleColors: Seq[String] = List(black, white, red, green, blue)
regexString: String = BLACK|WHITE|RED|GREEN|BLUE


주어진 문자를 다른 문자로 치환해야 할 때도 있음<br/>
정규 표현식을 만드는 것이 지루할 수 있으므로 translate 함수를 사용해 문자를 치환해보겠음<br/>
이 연산은 문자 단위로 이루어짐<br/>
교체 문자열에서 색인된 문자에 해당하는 모든 문자를 치환함<br/>

In [78]:
import org.apache.spark.sql.functions.translate

df.select(translate(col("Description"), "LEET", "1337"), col("Description")) // L=1, E=3, T=7로 치환됨
    .show(2, false)

+----------------------------------+----------------------------------+
|translate(Description, LEET, 1337)|Description                       |
+----------------------------------+----------------------------------+
|WHI73 HANGING H3AR7 7-1IGH7 HO1D3R|WHITE HANGING HEART T-LIGHT HOLDER|
|WHI73 M37A1 1AN73RN               |WHITE METAL LANTERN               |
+----------------------------------+----------------------------------+
only showing top 2 rows



import org.apache.spark.sql.functions.translate


처음 나타난 색상 이름을 추출하는 것과 같은 작업을 수행할 수도 있음<br/>

In [97]:
import org.apache.spark.sql.functions.regexp_extract

val regexString = simpleColors.map(_.toUpperCase).mkString("(", "|", ")")
df.select(
    regexp_extract(col("Description"), regexString, 1).alias("color.clean"), 
    col("Description")).show(25, false)

+-----------+-----------------------------------+
|color.clean|Description                        |
+-----------+-----------------------------------+
|WHITE      |WHITE HANGING HEART T-LIGHT HOLDER |
|WHITE      |WHITE METAL LANTERN                |
|           |CREAM CUPID HEARTS COAT HANGER     |
|           |KNITTED UNION FLAG HOT WATER BOTTLE|
|RED        |RED WOOLLY HOTTIE WHITE HEART.     |
|           |SET 7 BABUSHKA NESTING BOXES       |
|           |GLASS STAR FROSTED T-LIGHT HOLDER  |
|           |HAND WARMER UNION JACK             |
|RED        |HAND WARMER RED POLKA DOT          |
|           |ASSORTED COLOUR BIRD ORNAMENT      |
|           |POPPY'S PLAYHOUSE BEDROOM          |
|           |POPPY'S PLAYHOUSE KITCHEN          |
|           |FELTCRAFT PRINCESS CHARLOTTE DOLL  |
|           |IVORY KNITTED MUG COSY             |
|           |BOX OF 6 ASSORTED COLOUR TEASPOONS |
|           |BOX OF VINTAGE JIGSAW BLOCKS       |
|           |BOX OF VINTAGE ALPHABET BLOCKS     |


import org.apache.spark.sql.functions.regexp_extract
regexString: String = (BLACK|WHITE|RED|GREEN|BLUE)


*의문사항*<br/>
위에서 '처음 나타난' 색상 이름을 추출한다고 했는데 그게 무슨 뜻이지?<br/>
전체 로우에서 WHITE가 처음 나온 경우를 찾는다는 건가? (이건 아닌 듯)<br/>
각 로우에서 색깔 단어가 몇 개가 나올 수 있는데 그 중 첫 번째 색깔 단어를 추출하는 건가?<br/>
아니면 각 로우에서 첫 번째 단어를 보고 그게 색깔 단어이면 추출하는 건가?<br/>

*해결*<br/>
각 로우마다 앞에서부터 읽으면서 처음 찾은 색깔 단어를 추출하는 것임<br/>

regexp_extract의 3번째 argument인 groupIdx에 대한 설명은 [여기](https://stackoverflow.com/questions/58268864/groupidx-parameter-of-spark-regexp-extract-function)를 참고할 것<br/>

값 추출 없이 단순히 값의 존재 여부를 확인하고 싶을 때는 contains 메서드를 사용함<br/>
이 메서드는 인수로 입력된 값이 컬럼의 문자열에 존재하는지 불리언 타입으로 반환함<br/>

In [108]:
val containsBlack = col("Description").contains("BLACK")
val containsWhite = col("Description").contains("WHITE")
df.withColumn("hasSimpleColor", containsBlack.or(containsWhite))
    .where("hasSimpleColor")
    .select("Description").show(5, false)

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
+----------------------------------+
only showing top 5 rows



containsBlack: org.apache.spark.sql.Column = contains(Description, BLACK)
containsWhite: org.apache.spark.sql.Column = contains(Description, WHITE)


위 예제는 값을 2개만 사용하므로 간단해보이지만, 값의 개수가 늘어나면 복잡해짐<br/>

동적으로 인수의 개수가 변하는 상황을 스파크는 어떻게 처리할까?<br/>
값 목록을 인수로 변환해 함수에 전달할 때는 varargs라 불리는 스칼라 고유 기능을 활용함<br/>
이 기능을 사용해 임의 길이의 배열을 효율적으로 다룰 수 있음<br/>
예를 들어 select 메서드와 varargs를 함께 사용해 원하는 만큼 동적으로 컬럼을 생성할 수 있음<br/>

In [109]:
val simpleColors = Seq("black", "white", "red", "green", "blue")

val selectedColumns = simpleColors.map(color => {
    col("Description").contains(color.toUpperCase).alias(s"is_$color")
}) :+expr("*") // 이 값을 추가할 수도 있음

df.select(selectedColumns:_*).where(col("is_white").or(col("is_red")))
    .select("Description").show(3, false)

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
+----------------------------------+
only showing top 3 rows



simpleColors: Seq[String] = List(black, white, red, green, blue)
selectedColumns: Seq[org.apache.spark.sql.Column] = List(contains(Description, BLACK) AS `is_black`, contains(Description, WHITE) AS `is_white`, contains(Description, RED) AS `is_red`, contains(Description, GREEN) AS `is_green`, contains(Description, BLUE) AS `is_blue`, unresolvedstar())


# 6.6 날짜와 타임스탬프 데이터 타입 다루기

날짜와 시간은 프로그래밍 언어와 DB 분야의 변함없는 과제임<br/>
계속해서 시간대(timezone)를 확인해야 하며, 포맷이 올바르고 유효한지 확인해야 함<br/>
스파크는 이런 복잡함을 피하고자 두 가지 종류의 시간 관련 정보만 집중적으로 관리함<br/>
하나는 달력 형태의 날짜(date)이고, 다른 하나는 날짜와 시간 정보를 모두 가지는 타임스탬프(timestamp)임<br/>
스파크는 inferSchema 옵션이 활성화된 경우 날짜와 타임스탬프를 포함해 컬럼의 데이터 타입을 최대한 정확하게 식별하려 시도함<br/>
스파크는 특정 날짜 포맷을 명시하지 않아도 자체적으로 식별해 데이터를 읽을 수 있음<br/>
따라서 예제의 데이터셋이 잘 동작하는 것을 확인할 수 있음<br/>

날짜와 타임스탬프를 다루는 작업은 문자열을 다루는 작업과 관련이 있음<br/>
날짜나 시간을 문자열로 저장하고 런타임에 날짜 타입으로 변환하는 경우가 많기 때문임<br/>
DB나 구조적 데이터를 다룰 떄는 이러한 작업이 드물지만, 텍스트나 CSV 파일을 다룰 때는 많이 발생함<br/>
이에 대해 짧게 알아보겠음<br/>

*CAUTION*<br/>
날짜와 타임스탬프, 특히 시간대를 다룰 때는 많은 주의가 필요함<br/>
시간대 설정이 필요하면 스파크 SQL 설정의 spark.conf.sessionLocalTimeZone 속성을 로컬 시간대로 지정해 적용할 수 있음<br/>
시간대 포맷은 반드시 자바 TimeZone 포맷을 따라야 함<br/>

In [110]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



스파크는 날짜와 시간을 최대한 올바른 형태로 읽고자 노력함<br/>
만약 특이한 포맷의 날짜와 시간 데이터를 어쩔 수 없이 다뤄야 한다면 각 단계별로 어떤 데이터 타입과 포맷을 유지하는지 정확히 알고 transformation을 적용해야 함<br/>
TimestampType 클래스는 초 단위 정밀도까지만 지원함<br/>
그러므로 밀리세컨드나 마이크로세컨드 단위를 다룬다면 Long 데이터 타입으로 데이터를 변환해 처리하는 우회 정책을 사용해야 함<br/>
그 이상의 정밀도는 TimestampType으로 변환될 때 제거됨<br/>

스파크는 특정 시점에 데이터 포맷이 약간 특이하게 변할 수 있음<br/>
이러한 문제를 피하려면 파싱이나 변환 작업을 해야 함<br/>
스파크는 자바의 날짜와 타임스탬프를 사용해서 표준 체계를 따름<br/>
다음은 오늘 날짜와 현재 타임스탬프 값을 구하는 예제임<br/>

In [115]:
import org.apache.spark.sql.functions.{current_date, current_timestamp}

val dateDF = spark.range(10)
    .withColumn("today", current_date())
    .withColumn("now", current_timestamp())
dateDF.createOrReplaceTempView("dateTable")
spark.sql("SELECT * FROM dateTable").show()

+---+----------+--------------------+
| id|     today|                 now|
+---+----------+--------------------+
|  0|2022-01-03|2022-01-03 23:37:...|
|  1|2022-01-03|2022-01-03 23:37:...|
|  2|2022-01-03|2022-01-03 23:37:...|
|  3|2022-01-03|2022-01-03 23:37:...|
|  4|2022-01-03|2022-01-03 23:37:...|
|  5|2022-01-03|2022-01-03 23:37:...|
|  6|2022-01-03|2022-01-03 23:37:...|
|  7|2022-01-03|2022-01-03 23:37:...|
|  8|2022-01-03|2022-01-03 23:37:...|
|  9|2022-01-03|2022-01-03 23:37:...|
+---+----------+--------------------+



import org.apache.spark.sql.functions.{current_date, current_timestamp}
dateDF: org.apache.spark.sql.DataFrame = [id: bigint, today: date ... 1 more field]


In [116]:
dateDF.printSchema()

root
 |-- id: long (nullable = false)
 |-- today: date (nullable = false)
 |-- now: timestamp (nullable = false)



위 예제로 만들어진 DataFrame을 사용해 오늘을 기준으로 5일 전후의 날짜를 구해보겠음<br/>
date_sum 함수와 date_add 함수는 컬럼과 더하거나 뺄 날짜 수를 인수로 전달해야 함<br/>

In [117]:
import org.apache.spark.sql.functions.{date_add, date_sub}

dateDF.select(date_sub(col("today"), 5), date_add(col("today"), 5)).show(1)

+------------------+------------------+
|date_sub(today, 5)|date_add(today, 5)|
+------------------+------------------+
|        2021-12-29|        2022-01-08|
+------------------+------------------+
only showing top 1 row



import org.apache.spark.sql.functions.{date_add, date_sub}


두 날짜의 차이를 구하는 작업도 자주 발생함<br/>
두 날짜 사이의 일 수를 반환하는 datediff 함수를 사용해 이러한 작업을 수행할 수 있음<br/>
월별로 일수가 다르므로 날짜만 신경 쓰는 경우가 많지만, 두 날짜 사이의 개월 수를 반환하는 months_between 함수도 있음<br/>

In [118]:
import org.apache.spark.sql.functions.{datediff, months_between, to_date}

dateDF.withColumn("week_ago", date_sub(col("today"), 7))
    .select(datediff(col("week_ago"), col("today"))).show(1)

+-------------------------+
|datediff(week_ago, today)|
+-------------------------+
|                       -7|
+-------------------------+
only showing top 1 row



import org.apache.spark.sql.functions.{datediff, months_between, to_date}


In [119]:
dateDF.select(
    to_date(lit("2016-01-01")).alias("start"),
    to_date(lit("2017-05-22")).alias("end"))
    .select(months_between(col("start"), col("end"))).show(1)

+--------------------------------+
|months_between(start, end, true)|
+--------------------------------+
|                    -16.67741935|
+--------------------------------+
only showing top 1 row



위 예제에서 to_date 함수를 처음 소개했음<br/>
to_date 함수는 문자열을 날짜로 변환할 수 있으며, 필요에 따라 날짜 포맷도 함께 지정할 수 있음<br/>
함수의 날짜 포맷은 반드시 [자바의 SimpleDateFormat 클래스](https://bit.ly/2Mz21Qc)가 지원하는 포맷을 사용해야 함<br/>

In [120]:
import org.apache.spark.sql.functions.{to_date, lit}

spark.range(5).withColumn("date", lit("2017-01-01"))
    .select(to_date(col("date"))).show(1)

+-------------+
|to_date(date)|
+-------------+
|   2017-01-01|
+-------------+
only showing top 1 row



import org.apache.spark.sql.functions.{to_date, lit}


스파크는 날짜를 파싱할 수 없다면 에러 대신 null 값을 반환하므로 다단계 처리 파이프라인에서는 까다로울 수 있음<br/>
데이터 포맷이 지정된 데이터에서 또 다른 포맷의 데이터가 나타날 수 있기 때문<br/>
이해를 돕기 위해 년-월-일 형태가 아닌 년-일-월 형태의 날짜 포맷을 사용해보겠음<br/>
스파크는 날짜를 파싱할 수 없으므로 null 값을 반환함<br/>

In [121]:
dateDF.select(to_date(lit("2016-20-12")), to_date(lit("2017-12-11"))).show(1)

+-------------------+-------------------+
|to_date(2016-20-12)|to_date(2017-12-11)|
+-------------------+-------------------+
|               null|         2017-12-11|
+-------------------+-------------------+
only showing top 1 row



지정한 날짜 형식에 맞춰 데이터가 들어온다면 특별한 문제가 발생하지 않음<br/>
하지만 날짜 형식을 지키지 않은 데이터가 들어온다면 디버깅하기 매우 어려움<br/>
위 예제에서 두 번째 날짜(2017-12-11)가 의도한 날짜인 11월 12일 대신 12월 11일로 표시되고 있음<br/>
스파크는 날짜가 뒤섞여 있거나 데이터가 잘못되었는지 판단할 수 없으므로 오류를 발생시키지 않음<br/>

그럼 코드를 수정해나가면서 이런 문제를 완전히 회피할 수 있는 방법을 찾아보겠음<br/>
첫 번째 단계는 자바의 SimpleDateFormat 표준에 맞춰 날짜 포맷을 지정하는 것임<br/>

문제를 해결하기 위해 to_date 함수와 to_timestamp 함수를 사용함<br/>
to_date 함수는 필요에 따라 날짜 포맷을 지정할 수 있지만 to_timestamp 함수는 반드시 날짜 포맷을 지정해야 함<br/>

In [122]:
import org.apache.spark.sql.functions.to_date

val dateFormat = "yyyy-dd-MM"
val cleanDateDF = spark.range(1).select(
    to_date(lit("2017-12-11"), dateFormat).alias("date"),
    to_date(lit("2017-20-12"), dateFormat).alias("date2"))
cleanDateDF.createOrReplaceTempView("dateTable2")
spark.sql("SELECT * FROM dateTable2").show()

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+



import org.apache.spark.sql.functions.to_date
dateFormat: String = yyyy-dd-MM
cleanDateDF: org.apache.spark.sql.DataFrame = [date: date, date2: date]


항상 날짜 포맷을 지정해야 하는 to_timestamp 함수의 예제를 살펴보겠음<br/>

In [123]:
import org.apache.spark.sql.functions.to_timestamp

cleanDateDF.select(to_timestamp(col("date"), dateFormat)).show()

+------------------------------+
|to_timestamp(date, yyyy-dd-MM)|
+------------------------------+
|           2017-11-12 00:00:00|
+------------------------------+



import org.apache.spark.sql.functions.to_timestamp


올바른 포맷과 타입의 날짜나 타임스탬프를 사용한다면 날짜를 매우 쉽게 비교할 수 있음<br/>
날짜를 비교할 때는 날짜나 타임스탬프 타입을 사용하거나 yyyy-MM-dd 포맷에 맞는 문자열을 지정함<br/>

In [125]:
cleanDateDF.filter(col("date2") > lit("2017-12-12")).select(col("date2")).show()

+----------+
|     date2|
+----------+
|2017-12-20|
+----------+



# 6.7 null 값 다루기

**DateFrame에서 빠져 있거나 비어 있는 데이터를 표현할 때는 항상 null 값을 사용하는 것이 좋음<br/>
스파크에서는 빈 문자열이나 대체 값 대신 null 값을 사용해야 최적화를 수행할 수 있음**<br/>
DataFrame의 하위 패키지인 .na를 사용하는 것이 DataFrame에서 null 값을 다루는 기본 방식임<br/>
또한 연산을 수행하면서 스파크가 null 값을 제어하는 방법을 명시적으로 지정하는 몇 가지 함수도 있음<br/>
자세한 내용은 5.4.15절 '로우 정렬하기'와 6.3절 '불리언 데이터 타입 다루기'를 참고할 것<br/>

*CAUTION*<br/>
null은 모든 프로그래밍 언어의 도전 과제임<br/>
암시적으로 null 값을 사용하는 것보다 명시적으로 사용하는 것이 항상 좋음<br/>
이전 예제에서는 null 데이터 타입을 가지는 컬럼을 어떻게 만들 수 있는지 알아보았음<br/>
하지만 여기에도 함정이 존재함<br/>
null 값을 허용하지 않는 컬럼을 선언해도 **강제성**은 없음<br/>
그러므로 정의한 스키마의 모든 컬럼이 null 값을 허용하지 않는다고 해도 스파크는 이를 강제할 수 없으며 null 값을 컬럼에 넣을 수 있음<br/>
nullable 속성은 스파크 SQL 옵티마이저가 해당 컬럼을 제어하는 동작을 단순하게 도울 뿐임<br/>
만약 null 값이 없어야 하는 컬럼에 null 값이 존재한다면 부정확한 결과를 초래하거나 디버깅하기 어려운 특이한 오류를 만날 수 있음<br/>

null 값을 다루는 두 가지 방법이 있음<br/>
* 명시적으로 null 값을 제어하거나,
* 전역 또는 컬럼 단위로 null 값을 특정 값으로 채워 넣는 것임

이 2가지 방법을 알아보겠음<br/>

## 6.7.1 coalesce

스파크의 coalesce 함수는 인수로 지정한 여러 컬럼 중 null이 아닌 첫 번째 값을 반환함<br/>
모든 컬럼이 null이 아닌 값을 가지는 경우 첫 번째 컬럼의 값을 반환함<br/>

In [127]:
import org.apache.spark.sql.functions.coalesce

df.select(coalesce(col("Description"), col("CustomerID"))).show()

+---------------------------------+
|coalesce(Description, CustomerID)|
+---------------------------------+
|             WHITE HANGING HEA...|
|              WHITE METAL LANTERN|
|             CREAM CUPID HEART...|
|             KNITTED UNION FLA...|
|             RED WOOLLY HOTTIE...|
|             SET 7 BABUSHKA NE...|
|             GLASS STAR FROSTE...|
|             HAND WARMER UNION...|
|             HAND WARMER RED P...|
|             ASSORTED COLOUR B...|
|             POPPY'S PLAYHOUSE...|
|             POPPY'S PLAYHOUSE...|
|             FELTCRAFT PRINCES...|
|             IVORY KNITTED MUG...|
|             BOX OF 6 ASSORTED...|
|             BOX OF VINTAGE JI...|
|             BOX OF VINTAGE AL...|
|             HOME BUILDING BLO...|
|             LOVE BUILDING BLO...|
|             RECIPE BOX WITH M...|
+---------------------------------+
only showing top 20 rows



import org.apache.spark.sql.functions.coalesce


## 6.7.2 ifnull, nulllf, nvl, nvl2

coalesce 함수와 유사한 결과를 얻을 수 있는 몇 가지 SQL 함수가 있음<br/>

ifnull 함수는 첫 번째 값이 null이면 두 번째 값을 반환함<br/>
첫 번째 값이 null이 아니면 첫 번째 값을 반환함<br/>

반면 nullif 함수는 두 값이 같으면 null을 반환함<br/>
두 값이 다르면 첫 번째 값을 반환함<br/>

nvl 함수는 첫 번째 값이 null이면 두 번째 값을 반환함<br/>
첫 번째 값이 null이 아니면 첫 번째 값을 반환함<br/>

마지막으로 nvl2 함수는 첫 번째 값이 null이 아니면 두 번째 값을 반환함<br/>
그리고 첫 번째 값이 null이면 세 번째 인수로 지정된 값을 반환함 (예제에서는 else_value)<br/>

이 함수들은 DataFrame의 select 표현식으로 사용할 수 있음

## 6.7.3 drop

drop 메서드는 null 값을 가진 로우를 제거하는 가장 간단한 함수임<br/>
기본적으로 null 값을 가진 모든 로우를 제거함<br/>

In [128]:
df.na.drop()
df.na.drop("any")

res123: org.apache.spark.sql.DataFrame = [InvoiceNo: string, StockCode: string ... 6 more fields]


drop 메서드의 인수로 any를 지정한 경우 로우의 컬럼값 중 하나라도 null 값을 가지면 해당 로우를 제거함<br/>
all을 지정한 경우 모든 컬럼의 값이 null이거나 NaN인 경우에만 해당 로우를 제거함<br/>

In [129]:
df.na.drop("all")

res124: org.apache.spark.sql.DataFrame = [InvoiceNo: string, StockCode: string ... 6 more fields]


drop 메서드에 배열 형태의 컬럼을 인수로 전달해 적용할 수도 있음<br/>

In [130]:
df.na.drop("all", Seq("StockCode", "InvoiceNo"))

res125: org.apache.spark.sql.DataFrame = [InvoiceNo: string, StockCode: string ... 6 more fields]


## 6.7.4 fill

fill 함수를 사용해 하나 이상의 컬럼을 특정 값으로 채울 수 있음<br/>
채워 넣을 값과 컬럼 집합으로 구성된 맵을 인수로 사용함<br/>

예컨대 String 데이터 타입의 컬럼에 존재하는 null 값을 다른 값으로 채워 넣는 방법은 다음과 같음<br/>

In [131]:
df.na.fill("All Null values become this string")

res126: org.apache.spark.sql.DataFrame = [InvoiceNo: string, StockCode: string ... 6 more fields]


df.na.fill(5:Integer) 같은 방식을 사용해 Integer 타입의 컬럼에 존재하는 null 값을 다른 값으로 채워 넣을 수 있음<br/>
Double 데이터 타입의 컬럼에는 df.na.fill(5:Double) 같은 방식으로 사용할 수 있음<br/>
다수의 컬럼에 적용하고 싶다면 적용하고자 하는 컬럼명을 배열로 만들어 인수로 사용함<br/>

In [132]:
df.na.fill(5, Seq("StockCode", "InvoiceNo"))

res127: org.apache.spark.sql.DataFrame = [InvoiceNo: string, StockCode: string ... 6 more fields]


스칼라 Map 타입을 사용해 다수의 컬럼에 fill 메서드를 적용할 수도 있음<br/>
여기서 key는 컬럼명이며, value는 채우는 데 사용할 값임<br/>

In [133]:
val fillColValues = Map("StockCode" -> 5, "Description" -> "No Value")
df.na.fill(fillColValues)

fillColValues: scala.collection.immutable.Map[String,Any] = Map(StockCode -> 5, Description -> No Value)
res128: org.apache.spark.sql.DataFrame = [InvoiceNo: string, StockCode: string ... 6 more fields]


## 6.7.5 replace

drop 메서드와 fill 메서드 외에도 null 값을 유연하게 대처할 방법이 있음<br/>
조건에 따라 다른 값으로 대체(replace)하는 것임<br/>
replace 메서드를 사용하려면 변경하고자 하는 값과 원래 값의 데이터 타입이 같아야 함<br/>

In [134]:
df.na.replace("Description", Map("" -> "UNKNOWN"))

res129: org.apache.spark.sql.DataFrame = [InvoiceNo: string, StockCode: string ... 6 more fields]


# 6.8 정렬하기

5장에서 설명한 것처럼 asc_nulls_first, desc_nulls_first, asc_nulls_last, desc_nulls_last 함수를 사용해 DataFrame 정렬 시 null 값 표시 기준을 지정할 수 있음<br/>

# 6.9 복합 데이터 타입 다루기

복합 데이터 타입을 사용하면 해결하려는 문제에 더욱 적합한 방식으로 데이터를 구성하고 구조화할 수 있음<br/>
복합 데이터 타입에는 **struct, array, map**이 있음<br/>

## 6.9.1 구조체

구조체는 DataFrame 내부의 DataFrame으로 생각할 수 있음<br/>
쿼리문에서 다수의 컬럼을 괄호로 묶어 구조체로 만들 수 있음<br/>

In [137]:
import org.apache.spark.sql.functions.struct

val complexDF = df.select(struct("Description", "InvoiceNo").alias("complex"))
complexDF.createOrReplaceTempView("complexDF")
spark.sql("SELECT * FROM complexDF").show(false)

+---------------------------------------------+
|complex                                      |
+---------------------------------------------+
|{WHITE HANGING HEART T-LIGHT HOLDER, 536365} |
|{WHITE METAL LANTERN, 536365}                |
|{CREAM CUPID HEARTS COAT HANGER, 536365}     |
|{KNITTED UNION FLAG HOT WATER BOTTLE, 536365}|
|{RED WOOLLY HOTTIE WHITE HEART., 536365}     |
|{SET 7 BABUSHKA NESTING BOXES, 536365}       |
|{GLASS STAR FROSTED T-LIGHT HOLDER, 536365}  |
|{HAND WARMER UNION JACK, 536366}             |
|{HAND WARMER RED POLKA DOT, 536366}          |
|{ASSORTED COLOUR BIRD ORNAMENT, 536367}      |
|{POPPY'S PLAYHOUSE BEDROOM , 536367}         |
|{POPPY'S PLAYHOUSE KITCHEN, 536367}          |
|{FELTCRAFT PRINCESS CHARLOTTE DOLL, 536367}  |
|{IVORY KNITTED MUG COSY , 536367}            |
|{BOX OF 6 ASSORTED COLOUR TEASPOONS, 536367} |
|{BOX OF VINTAGE JIGSAW BLOCKS , 536367}      |
|{BOX OF VINTAGE ALPHABET BLOCKS, 536367}     |
|{HOME BUILDING BLOCK WORD, 536367}     

import org.apache.spark.sql.functions.struct
complexDF: org.apache.spark.sql.DataFrame = [complex: struct<Description: string, InvoiceNo: string>]


복합 데이터 타입을 가진 DataFrame을 만들어보았음<br/>
이를 다른 DataFrame을 조회하는 것과 동일하게 사용할 수 있음<br/>
유일한 차이점은 문법에 점(.)을 사용하거나 getField 메서드를 사용한다는 것임<br/>

In [138]:
complexDF.select("complex.Description")
complexDF.select(col("complex").getField("Description")).show(false)

+-----------------------------------+
|complex.Description                |
+-----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER |
|WHITE METAL LANTERN                |
|CREAM CUPID HEARTS COAT HANGER     |
|KNITTED UNION FLAG HOT WATER BOTTLE|
|RED WOOLLY HOTTIE WHITE HEART.     |
|SET 7 BABUSHKA NESTING BOXES       |
|GLASS STAR FROSTED T-LIGHT HOLDER  |
|HAND WARMER UNION JACK             |
|HAND WARMER RED POLKA DOT          |
|ASSORTED COLOUR BIRD ORNAMENT      |
|POPPY'S PLAYHOUSE BEDROOM          |
|POPPY'S PLAYHOUSE KITCHEN          |
|FELTCRAFT PRINCESS CHARLOTTE DOLL  |
|IVORY KNITTED MUG COSY             |
|BOX OF 6 ASSORTED COLOUR TEASPOONS |
|BOX OF VINTAGE JIGSAW BLOCKS       |
|BOX OF VINTAGE ALPHABET BLOCKS     |
|HOME BUILDING BLOCK WORD           |
|LOVE BUILDING BLOCK WORD           |
|RECIPE BOX WITH METAL HEART        |
+-----------------------------------+
only showing top 20 rows



\* 문자를 사용해 모든 값을 조회할 수 있으며, 모든 컬럼을 DataFrame의 최상위 수준으로 끌어올릴 수 있음<br/>

In [141]:
complexDF.select("complex.*")
    .show(false)

+-----------------------------------+---------+
|Description                        |InvoiceNo|
+-----------------------------------+---------+
|WHITE HANGING HEART T-LIGHT HOLDER |536365   |
|WHITE METAL LANTERN                |536365   |
|CREAM CUPID HEARTS COAT HANGER     |536365   |
|KNITTED UNION FLAG HOT WATER BOTTLE|536365   |
|RED WOOLLY HOTTIE WHITE HEART.     |536365   |
|SET 7 BABUSHKA NESTING BOXES       |536365   |
|GLASS STAR FROSTED T-LIGHT HOLDER  |536365   |
|HAND WARMER UNION JACK             |536366   |
|HAND WARMER RED POLKA DOT          |536366   |
|ASSORTED COLOUR BIRD ORNAMENT      |536367   |
|POPPY'S PLAYHOUSE BEDROOM          |536367   |
|POPPY'S PLAYHOUSE KITCHEN          |536367   |
|FELTCRAFT PRINCESS CHARLOTTE DOLL  |536367   |
|IVORY KNITTED MUG COSY             |536367   |
|BOX OF 6 ASSORTED COLOUR TEASPOONS |536367   |
|BOX OF VINTAGE JIGSAW BLOCKS       |536367   |
|BOX OF VINTAGE ALPHABET BLOCKS     |536367   |
|HOME BUILDING BLOCK WORD           |536

## 6.9.2 배열

배열을 정의하기 위해 한 가지 사례를 살펴보겠음<br/>
데이터에서 Description 컬럼의 모든 단어를 하나의 로우로 변환할 것임<br/>

우선 Description 컬럼을 복합 데이터 타입인 배열로 변환하겠음<br/>

### split

배열로 변환하려면 split 함수를 사용함<br/>
split 함수에 구분자(delimiter)를 인수로 전달해 배열로 변환함<br/>

In [142]:
import org.apache.spark.sql.functions.split

df.select(split(col("Description"), " ")).show(2, false)

+----------------------------------------+
|split(Description,  , -1)               |
+----------------------------------------+
|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|
|[WHITE, METAL, LANTERN]                 |
+----------------------------------------+
only showing top 2 rows



import org.apache.spark.sql.functions.split


split 함수는 스파크에서 복합 데이터 타입을 마치 또 다른 컬럼처럼 다룰 수 있는 매우 강력한 기능임<br/>
파이썬과 유사한 문법을 사용해 배열값을 조회할 수 있음<br/>

In [143]:
df.select(split(col("Description"), " ").alias("array_col"))
    .selectExpr("array_col[0]").show(2, false)

+------------+
|array_col[0]|
+------------+
|WHITE       |
|WHITE       |
+------------+
only showing top 2 rows



### 배열의 길이

배열의 크기(size)를 조회해 배열의 길이를 알 수 있음<br/>

In [144]:
import org.apache.spark.sql.functions.size

df.select(size(split(col("Description"), " "))).show(2, false) // 5와 3 출력

+-------------------------------+
|size(split(Description,  , -1))|
+-------------------------------+
|5                              |
|3                              |
+-------------------------------+
only showing top 2 rows



import org.apache.spark.sql.functions.size


### array_contains

array_contains 함수를 사용해 배열에 특정 값이 존재하는지 확인할 수 있음<br/>

In [147]:
import org.apache.spark.sql.functions.array_contains

df.select(array_contains(split(col("Description"), " "), "WHITE")).show(5, false)

+------------------------------------------------+
|array_contains(split(Description,  , -1), WHITE)|
+------------------------------------------------+
|true                                            |
|true                                            |
|false                                           |
|false                                           |
|true                                            |
+------------------------------------------------+
only showing top 5 rows



import org.apache.spark.sql.functions.array_contains


하지만 이 함수로 우리 시나리오를 완성할 수 없음<br/>
복합 데이터 타입의 배열에 존재하는 모든 값을 로우로 변환하려면 explode 함수를 사용함<br/>

### explode

explode 함수는 배열 타입의 컬럼을 입력받음<br/>
그리고 입력된 컬럼의 배열값에 포함된 모든 값을 로우로 변환함<br/>
나머지 컬럼값은 중복되어 표시됨<br/>

In [148]:
import org.apache.spark.sql.functions.{split, explode}

df.withColumn("splitted", split(col("Description"), " "))
    .withColumn("exploded", explode(col("splitted")))
    .select("Description", "InvoiceNo", "splitted", "exploded").show(false)

+-----------------------------------+---------+------------------------------------------+--------+
|Description                        |InvoiceNo|splitted                                  |exploded|
+-----------------------------------+---------+------------------------------------------+--------+
|WHITE HANGING HEART T-LIGHT HOLDER |536365   |[WHITE, HANGING, HEART, T-LIGHT, HOLDER]  |WHITE   |
|WHITE HANGING HEART T-LIGHT HOLDER |536365   |[WHITE, HANGING, HEART, T-LIGHT, HOLDER]  |HANGING |
|WHITE HANGING HEART T-LIGHT HOLDER |536365   |[WHITE, HANGING, HEART, T-LIGHT, HOLDER]  |HEART   |
|WHITE HANGING HEART T-LIGHT HOLDER |536365   |[WHITE, HANGING, HEART, T-LIGHT, HOLDER]  |T-LIGHT |
|WHITE HANGING HEART T-LIGHT HOLDER |536365   |[WHITE, HANGING, HEART, T-LIGHT, HOLDER]  |HOLDER  |
|WHITE METAL LANTERN                |536365   |[WHITE, METAL, LANTERN]                   |WHITE   |
|WHITE METAL LANTERN                |536365   |[WHITE, METAL, LANTERN]                   |METAL   |


import org.apache.spark.sql.functions.{split, explode}


## 6.9.3 맵

맵은 map 함수와 컬럼의 키-값 쌍을 이용해 생성함<br/>
그리고 배열과 동일한 방법으로 값을 선택할 수 있음<br/>

In [150]:
import org.apache.spark.sql.functions.map

df.select(map(col("Description"), col("InvoiceNo")).alias("complex_map")).show(2, false)

+----------------------------------------------+
|complex_map                                   |
+----------------------------------------------+
|{WHITE HANGING HEART T-LIGHT HOLDER -> 536365}|
|{WHITE METAL LANTERN -> 536365}               |
+----------------------------------------------+
only showing top 2 rows



import org.apache.spark.sql.functions.map


적합한 키를 사용해 데이터를 조회할 수 있으며, 해당 키가 없다면 null 값을 반환함<br/>

In [151]:
df.select(map(col("Description"), col("InvoiceNo")).alias("complex_map"))
    .selectExpr("complex_map['WHITE METAL LANTERN']").show(2, false)

+--------------------------------+
|complex_map[WHITE METAL LANTERN]|
+--------------------------------+
|null                            |
|536365                          |
+--------------------------------+
only showing top 2 rows



*의문사항*<br/>
위 예제에서 complex_map과 complex_map\[WHITE METAL LANTERN\] 컬럼이 모두 표시되어야 할 것 같은데 왜 그렇지 않은지 의문이 생김<br/>

아래 코드 에러 메시지를 보면 \[complex_map\]이 input columns임을 알 수 있음<br/>
즉, select에 map이라는 복합 데이터 타입을 인풋으로 주는 경우, complex_map이 단순히 표시해야 할 컬럼으로 처리되는 게 아님<br/>
select에 map을 인풋으로 주고 그냥 show를 하면 key-value 매핑이 내용으로 표시가 되고, select에 이어 key를 지정하면 내용으로 value가 표시되는 것<br/>

In [158]:
// 에러 메시지에 유의
df.select(map(col("Description"), col("InvoiceNo")).alias("complex_map"))
    .selectExpr("complex_map['WHITE METAL LANTERN']", "StockCode").show(2, false)

org.apache.spark.sql.AnalysisException:  cannot resolve '`StockCode`' given input columns: [complex_map]; line 1 pos 0;

map 타입은 분해해서 컬럼으로 변환할 수 있음<br/>

In [162]:
// map이 있을 때 각각의 key-value 쌍을 다 로우로 빼서 보여주는 방법 -> select에 map을 넣고 selectExpr에 explode(<map 이름>)을 넣음
df.select(map(col("Description"), col("InvoiceNo")).alias("complex_map"))
    .selectExpr("explode(complex_map)").show(false)

+-----------------------------------+------+
|key                                |value |
+-----------------------------------+------+
|WHITE HANGING HEART T-LIGHT HOLDER |536365|
|WHITE METAL LANTERN                |536365|
|CREAM CUPID HEARTS COAT HANGER     |536365|
|KNITTED UNION FLAG HOT WATER BOTTLE|536365|
|RED WOOLLY HOTTIE WHITE HEART.     |536365|
|SET 7 BABUSHKA NESTING BOXES       |536365|
|GLASS STAR FROSTED T-LIGHT HOLDER  |536365|
|HAND WARMER UNION JACK             |536366|
|HAND WARMER RED POLKA DOT          |536366|
|ASSORTED COLOUR BIRD ORNAMENT      |536367|
|POPPY'S PLAYHOUSE BEDROOM          |536367|
|POPPY'S PLAYHOUSE KITCHEN          |536367|
|FELTCRAFT PRINCESS CHARLOTTE DOLL  |536367|
|IVORY KNITTED MUG COSY             |536367|
|BOX OF 6 ASSORTED COLOUR TEASPOONS |536367|
|BOX OF VINTAGE JIGSAW BLOCKS       |536367|
|BOX OF VINTAGE ALPHABET BLOCKS     |536367|
|HOME BUILDING BLOCK WORD           |536367|
|LOVE BUILDING BLOCK WORD           |536367|
|RECIPE BO

# 6.10 JSON 다루기

스파크는 JSON 데이터를 다루기 위한 몇 가지 고유 기능을 지원함<br/>
스파크에서는 문자열 형태의 JSON을 직접 조작할 수 있으며, JSON을 파싱하거나 JSON 객체로 만들 수 있음<br/>
다음은 JSON 컬럼을 생성하는 예제임<br/>

In [163]:
val jsonDF = spark.range(1).selectExpr("""
    '{"myJSONKey" : {"myJSONValue" : [1, 2, 3]}}' as jsonString""") // 표현식이 한 줄을 넘어갈 땐 " "를 쓰는 게 아니라 """ """를 씀

jsonDF: org.apache.spark.sql.DataFrame = [jsonString: string]


get_json_object 함수로 JSON 객체(딕셔너리나 배열)를 인라인 쿼리로 조회할 수 있음<br/>
중첩이 없는 단일 수준의 JSON 객체라면 json_tuple을 사용할 수도 있음<br/>

In [165]:
import org.apache.spark.sql.functions.{get_json_object, json_tuple}

jsonDF.select(
    get_json_object(col("jsonString"), "$.myJSONKey.myJSONValue[1]") as "column",
    json_tuple(col("jsonString"), "myJSONKey")).show(false)

+------+-----------------------+
|column|c0                     |
+------+-----------------------+
|2     |{"myJSONValue":[1,2,3]}|
+------+-----------------------+



import org.apache.spark.sql.functions.{get_json_object, json_tuple}


to_json 함수를 사용해 StructType을 JSON 문자열로 변경할 수 있음<br/>

In [167]:
import org.apache.spark.sql.functions.to_json

df.selectExpr("(InvoiceNo, Description) as myStruct")
    .select(to_json(col("myStruct")))
    .show(false)

+--------------------------------------------------------------------------+
|to_json(myStruct)                                                         |
+--------------------------------------------------------------------------+
|{"InvoiceNo":"536365","Description":"WHITE HANGING HEART T-LIGHT HOLDER"} |
|{"InvoiceNo":"536365","Description":"WHITE METAL LANTERN"}                |
|{"InvoiceNo":"536365","Description":"CREAM CUPID HEARTS COAT HANGER"}     |
|{"InvoiceNo":"536365","Description":"KNITTED UNION FLAG HOT WATER BOTTLE"}|
|{"InvoiceNo":"536365","Description":"RED WOOLLY HOTTIE WHITE HEART."}     |
|{"InvoiceNo":"536365","Description":"SET 7 BABUSHKA NESTING BOXES"}       |
|{"InvoiceNo":"536365","Description":"GLASS STAR FROSTED T-LIGHT HOLDER"}  |
|{"InvoiceNo":"536366","Description":"HAND WARMER UNION JACK"}             |
|{"InvoiceNo":"536366","Description":"HAND WARMER RED POLKA DOT"}          |
|{"InvoiceNo":"536367","Description":"ASSORTED COLOUR BIRD ORNAMENT"}      |

import org.apache.spark.sql.functions.to_json


to_json 함수에 JSON 데이터 소스와 동일한 형태의 딕셔너리(맵)를 파라미터로 사용할 수 있음<br/>
그리고 from_json 함수를 사용해 JSON 문자열을 다시 객체로 변환할 수 있음<br/>
from_json 함수는 파라미터로 반드시 스키마를 지정해야 함<br/>
필요에 따라 맵 데이터 타입의 옵션을 인수로 지정할 수도 있음<br/>

In [168]:
import org.apache.spark.sql.functions.from_json
import org.apache.spark.sql.types._

val parseSchema = new StructType(Array(
    new StructField("InvoiceNo", StringType, true),
    new StructField("Description", StringType, true)))
df.selectExpr("(InvoiceNo, Description) as myStruct")
    .select(to_json(col("myStruct")).alias("newJSON"))
    .select(from_json(col("newJSON"), parseSchema), col("newJSON")).show(false)

+---------------------------------------------+--------------------------------------------------------------------------+
|from_json(newJSON)                           |newJSON                                                                   |
+---------------------------------------------+--------------------------------------------------------------------------+
|{536365, WHITE HANGING HEART T-LIGHT HOLDER} |{"InvoiceNo":"536365","Description":"WHITE HANGING HEART T-LIGHT HOLDER"} |
|{536365, WHITE METAL LANTERN}                |{"InvoiceNo":"536365","Description":"WHITE METAL LANTERN"}                |
|{536365, CREAM CUPID HEARTS COAT HANGER}     |{"InvoiceNo":"536365","Description":"CREAM CUPID HEARTS COAT HANGER"}     |
|{536365, KNITTED UNION FLAG HOT WATER BOTTLE}|{"InvoiceNo":"536365","Description":"KNITTED UNION FLAG HOT WATER BOTTLE"}|
|{536365, RED WOOLLY HOTTIE WHITE HEART.}     |{"InvoiceNo":"536365","Description":"RED WOOLLY HOTTIE WHITE HEART."}     |
|{536365, SET 7 

import org.apache.spark.sql.functions.from_json
import org.apache.spark.sql.types._
parseSchema: org.apache.spark.sql.types.StructType = StructType(StructField(InvoiceNo,StringType,true), StructField(Description,StringType,true))


# 6.11 사용자 정의 함수

스파크의 가장 강력한 기능 중 하나는 사용자 정의 함수(user defined function, UDF)를 사용할 수 있다는 것임<br/>
UDF는 파이썬이나 스칼라 그리고 외부 라이브러리를 사용해 사용자가 원하는 형태로 transformation을 만들 수 있게 함<br/>
UDF는 하나 이상의 컬럼을 입력으로 받고, 반환할 수 있음<br/>
스파크 UDF는 여러 가지 프로그래밍 언어로 개발할 수 있으므로 매우 강력함<br/>
UDF는 레코드별로 데이터를 처리하는 함수이기 때문에 독특한 포맷이나 도메인에 특화된 언어를 사용하지 않음<br/>
이러한 UDF는 기본적으로 특정 SparkSession이나 Context에서 사용할 수 있도록 임시 함수 형태로 등록됨<br/>

스칼라, 파이썬, 자바로 UDF를 개발할 수 있음<br/>
*하지만 언어별로 성능에 영향을 미칠 수 있으므로 주의해야 함*<br/>
이 부분을 설명하기 위해 UDF를 생성해서 스파크에 등록할 것임<br/>
그리고 생성된 UDF를 사용해 코드를 실행하는 과정에서 정확히 무슨 일이 발생하는지 알아보겠음<br/>

첫 번째로 실제 함수가 필요함<br/>
예제로 사용할 UDF를 만들어보겠음<br/>
숫자를 입력받아 세제곱 연산을 하는 power3 함수를 아래와 같이 개발할 수 있음<br/>

In [170]:
val udfExampleDF = spark.range(5).toDF("num")
def power3(number:Double):Double = number * number * number
power3(2.0)

udfExampleDF: org.apache.spark.sql.DataFrame = [num: bigint]
power3: (number: Double)Double
res163: Double = 8.0


위 예제를 실행하면 power3 함수가 정상 동작하는 것을 알 수 있음<br/>
즉, 함수를 정의해 입력값을 원하는 결과로 만들어낼 수 있음<br/>
입력 조건을 까다롭게 만들어 조금 더 개선할 수 있음<br/>
입력값을 특정 데이터 타입으로 강제하고 null 값을 입력하지 못하게 만들어야 함<br/>

이제 함수를 만드록 테스트를 완료했으므로 모든 워커 노드에서 생성된 함수를 사용할 수 있도록 스파크에 등록한 차례임<br/>
스파크는 드라이버에서 함수를 직렬화하고 네트워크를 통해 모든 executor 프로세스로 전달함<br/>
이 과정은 사용하는 언어와 관계없이 발생함<br/>

**함수를 개발한 언어에 따라 근본적으로 동작하는 방식이 달라짐<br/>
스칼라나 자바로 함수를 작성했다면 JVM 환경에서만 사용할 수 있음<br/>
따라서 스파크 내장 함수가 제공하는 코드 생성 기능의 장점을 활용할 수 없어 약간의 성능 저하가 발생함<br/>
그리고 많은 객체를 생성하거나 사용해도 성능 문제가 발생함<br/>
성능 최적화는 19장 '성능 튜닝'에서 자세히 알아보겠음**<br/>

**파이썬으로 함수를 작성했다면 매우 다르게 동작함<br/>
스파크는 워커 노드에 파이썬 프로세스를 실행하고 파이썬이 이해할 수 있는 포맷으로 모든 데이터를 직렬화(앞서 JVM 사용 언어에도 존재했던 부분)함<br/>
그리고 파이썬 프로세스에 있는 데이터의 로우마다 함수를 실행하고 마지막으로 JVM과 스파크에 처리 결과를 반환함**<br/>

*CAUTION*<br/>
파이썬 프로세스를 시작하는 부하도 크지만 진짜 부하는 파이썬으로 데이터를 전달하기 위해 직렬화하는 과정에서 발생함<br/>
이 특성은 두 가지 문제점을 만들어냄<br/>
첫째, 직렬화에 큰 부하가 발생함<br/>
둘째, 데이터가 파이썬으로 전달되면 스파크에서 워커 메모리를 관리할 수 없음<br/>
그러므로 JVM과 파이썬이 동일한 머신에서 메모리 경합을 하면 자원에 제약이 생겨 워커가 비정상적으로 종료될 가능성이 있음<br/>
스칼라로 함수를 개발하는 것은 시간이 오래 걸리지 않으며, 무엇보다 파이썬에서도 사용할 수 있으므로 자바나 스칼라로 사용자 정의 함수를 작성하는 것이 좋음<br/>

*의문사항*<br/>
파이썬으로 작성한 함수를 실행할 때의 동작 방식을 설명하며 CAUTION에서 'JVM과 파이썬이 동일한 머신에서 메모리 경합'을 하는 상황을 얘기했음<br/>
파이썬으로 작성한 함수를 실행할 때 왜 그런 상황이 되는지?<br/>

파이썬 UDF 처리 과정을 보면 우선 드라이버에서 함수 직렬화 후 워커에 전달함<br/>
다음으로 워커에서 executor 프로세스가 파이썬 프로세스를 실행 후 파이썬 프로세스에 데이터를 전송함<br/>
마지막으로 파이썬 프로세스가 실행을 마치면 처리 결과를 executor 프로세스에 반환함<br/>
여기서 볼 수 있듯 워커에는 executor process와 워커 파이썬 프로세스가 있게 되는데, executor process는 JVM에서 동작함<br/>
(스파크는 스칼라로 작성되었고 스칼라는 JVM 기반 언어이므로)<br/>
따라서 워커를 보면 JVM과 파이썬이 동일 머신(워커)에서 메모리 경합을 하게 됨<br/>

UDF를 실행해보겠음<br/>
먼저 DataFrame에서 사용할 수 있도록 함수를 등록함<br/>

In [172]:
import org.apache.spark.sql.functions.udf

val power3udf = udf(power3(_:Double):Double)

import org.apache.spark.sql.functions.udf
power3udf: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($$$82b5b23cea489b2712a1db46c77e458$$$$w$$Lambda$4886/0x0000000841a0f840@4c79f755,DoubleType,List(Some(class[value[0]: double])),Some(class[value[0]: double]),None,false,true)


이제 power3 함수를 DataFrame의 다른 함수와 동일한 방법으로 사용할 수 있음<br/>

In [173]:
udfExampleDF.select(power3udf(col("num"))).show()

+--------+
|UDF(num)|
+--------+
|     0.0|
|     1.0|
|     8.0|
|    27.0|
|    64.0|
+--------+



아직까지는 사용자 정의 함수를 DataFrame에서만 사용할 수 있음<br/>
문자열 표현식에서는 사용할 수 없음<br/>
하지만 사용자 정의 함수를 스파크 SQL 함수로 등록하면 이야기는 달라짐<br/>
SQL 함수로 등록하면 모든 프로그래밍 언어와 SQL에서 사용자 정의 함수를 사용할 수 있음<br/>

스칼라를 사용해 사용자 정의 함수를 등록해보겠음<br/>

In [174]:
spark.udf.register("power3", power3(_:Double):Double)
udfExampleDF.selectExpr("power3(num)").show(false)

+-----------+
|power3(num)|
+-----------+
|0.0        |
|1.0        |
|8.0        |
|27.0       |
|64.0       |
+-----------+



사용자 정의 함수를 스파크 SQL 함수로 등록했기 때문에 스칼라로 개발된 사용자 정의 함수를 파이썬에서 우회적으로 사용할 수 있음<br/>
그 이유는 DataFrame에 스파크 SQL 함수나 SQL 표현식을 사용할 수 있기 때문임<br/>
하지만 사용자 정의 함수를 DataFrame 함수 대신 SQL 표현식으로 사용해야 함<br/>

In [175]:
/*
# 파이썬 코드
udfExampleDF.selectExpr("power3(num)").show(false)
# 스칼라로 등록된 사용자 정의 함수 사용
*/

# 6.12 Hive UDF

하이브 문법을 사용해서 만든 UDF/UDAF도 사용할 수 있음<br/>
이렇게 하려면 SparkSession을 생성할 때 SparkSession.builder().enableHiveSupport()를 명시해 반드시 하이브 지원 기능을 활성화해야 함<br/>
하이브 지원이 활성화되면 SQL로 UDF를 등록할 수 있음<br/>
사전에 컴파일된 스칼라와 자바 패키지에서만 지원되므로 라이브러리 의존성을 명시해야 함<br/>

# 6.13 정리

이 장에서는 어떻게 스파크 SQL을 사용 목적에 맞게 확장할 수 있는지 알아보았음<br/>
소수만 이해하는 형식이나 도메인에 특화된 언어가 아닌, 스파크를 사용하지 않고도 테스트 및 유지 보수할 수 있는 간단한 함수만으로도 스파크 SQL을 확장할 수 있었음<br/>
스파크 SQL은 복잡한 비즈니스 로직을 구현하는 데 사용할 수 있는 강력한 기능임<br/>
스파크 SQL을 사용해 로컬 머신의 5줄짜리 데이터부터 100개의 노드로 구성된 클러스터의 테라바이트급 데이터까지 모두 처리할 수 있음<br/>